In [1]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [2]:
import tensorflow as tf

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Force TensorFlow to use GPU
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

Num GPUs Available:  1


In [6]:
import pandas as pd
import joblib
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display

# Load the saved model from Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify the paths to your model and scaler on Google Drive
model_path = '/content/drive/My Drive/GradientBoosting.pkl'
scaler_path = '/content/drive/My Drive/scaler.pkl'

# Load the model and scaler
model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

# Define the feature columns
feature_columns = [
    'potential', 'age', 'shooting', 'passing', 'physic', 'movement_reactions',
    'preferred_foot_left', 'preferred_foot_right'
]

# Function to preprocess input data
def preprocess_input(preferred_foot, potential, age, shooting, passing, physic, movement_reactions):
    # One-hot encode the 'preferred_foot' field
    preferred_foot_left = 1 if preferred_foot == 'left' else 0
    preferred_foot_right = 1 if preferred_foot == 'right' else 0

    # Create a DataFrame
    input_data = {
        'potential': [potential],
        'age': [age],
        'shooting': [shooting],
        'passing': [passing],
        'physic': [physic],
        'movement_reactions': [movement_reactions],
        'preferred_foot_left': [preferred_foot_left],
        'preferred_foot_right': [preferred_foot_right]
    }

    df = pd.DataFrame(input_data)

    # Convert to sparse matrix
    sparse_matrix = csr_matrix(df.astype(pd.SparseDtype("float", 0)).sparse.to_coo())

    # Standardize features using the same scaler used during training
    X_scaled = scaler.transform(sparse_matrix)

    return X_scaled

# Function to handle prediction and display result
def predict_rating(preferred_foot, potential, age, shooting, passing, physic, movement_reactions):
    X_input = preprocess_input(preferred_foot, potential, age, shooting, passing, physic, movement_reactions)
    prediction = model.predict(X_input)[0]
    print(f'Predicted Overall Rating: {prediction:.2f}')

# Create interactive widgets for input
preferred_foot_widget = widgets.Dropdown(options=['left', 'right'], description='Preferred Foot:')
potential_widget = widgets.IntSlider(min=50, max=100, value=80, description='Potential:')
age_widget = widgets.IntSlider(min=16, max=40, value=25, description='Age:')
shooting_widget = widgets.IntSlider(min=50, max=100, value=70, description='Shooting:')
passing_widget = widgets.IntSlider(min=50, max=100, value=70, description='Passing:')
physic_widget = widgets.IntSlider(min=50, max=100, value=70, description='Physic:')
movement_reactions_widget = widgets.IntSlider(min=50, max=100, value=70, description='Movement Reactions:')

# Display widgets
display(preferred_foot_widget, potential_widget, age_widget, shooting_widget, passing_widget, physic_widget, movement_reactions_widget)

# Predict button
predict_button = widgets.Button(description='Predict')
display(predict_button)

# Event handler for button click
def on_predict_button_clicked(b):
    clear_output()
    predict_rating(preferred_foot_widget.value, potential_widget.value, age_widget.value,
                   shooting_widget.value, passing_widget.value, physic_widget.value,
                   movement_reactions_widget.value)

predict_button.on_click(on_predict_button_clicked)


Mounted at /content/drive


Dropdown(description='Preferred Foot:', options=('left', 'right'), value='left')

IntSlider(value=80, description='Potential:', min=50)

IntSlider(value=25, description='Age:', max=40, min=16)

IntSlider(value=70, description='Shooting:', min=50)

...

IntSlider(value=70, description='Physic:', min=50)

IntSlider(value=70, description='Movement Reactions:', min=50)

Button(description='Predict', style=ButtonStyle())

NameError: name 'clear_output' is not defined

NameError: name 'clear_output' is not defined

In [ ]:
!streamlit run app.py